In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [37]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

df = spark.read.csv('sales_info.csv', header=True, inferSchema=True)
df.head(5)

[Row(Company='GOOG', Person='Sam', Sales=200.0),
 Row(Company='GOOG', Person='Charlie', Sales=120.0),
 Row(Company='GOOG', Person='Frank', Sales=340.0),
 Row(Company='MSFT', Person='Tina', Sales=600.0),
 Row(Company='MSFT', Person='Amy', Sales=124.0)]

In [6]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [9]:
df.groupBy("Company").count().show()


+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [10]:
#agg({col}:'function')
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [15]:
df.select(avg('Sales').alias('Avg_Sales')).show()

+-----------------+
|        Avg_Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [22]:
df.select(stddev('Sales').alias('Std')).select(format_number('Std',2)).show()

+---------------------+
|format_number(Std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [24]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [28]:
df.na.drop().show()
df.na.drop(thresh=2).show() #only drop if have 2 non-null values
df.na.drop(how='all').show() #only drop if all are null
df.na.drop(subset=['Sales']).show() #drop if that column has a null val
df.na.fill('None', subset=['Sales']).show()




AttributeError: 'DataFrameNaFunctions' object has no attribute 'show'

In [51]:
#fill the mean_value into null places
#df.select(stddev('Sales').alias('Std')).select(format_number('Std',2)).show()


mean_sales = df.select(mean('Sales').alias('mean_sales')).select(format_number('mean_sales',2)).collect()
# mean_sales[0] #Row(avg(Sales)=360.5833333333333)
mean_sales = mean_sales[0][0]

In [53]:


df.na.fill(mean_sales, ['Sales']).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

